In [1]:
# ! pipenv run jupyter nbconvert --to python get_all_txs.ipynb
# !uv pip install dune_client

In [2]:
print('get all txs')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import google_bq_utils as bqu
sys.path.pop()

import time
import datetime

import numpy as np
import pandas as pd
import os

get all txs


In [3]:
col_list = [
        'dt','blockchain','name','layer','chain_id'
        , 'num_raw_txs', 'num_success_txs','num_qualified_txs','source'
        ]
query_name = 'dune_all_txs'
query_gas_name = 'dune_all_gas'
query_fees_name = 'dune_all_fees'
rerun_hrs = 4

In [4]:
# Run Config
run_all_txs = 1 #0
run_gas = 1 #0
run_metrics = 1 #0
run_chain_level = 1 #0

In [ ]:
trailing_days = 28 #45
ending_days = 0
single_chain = 'none' ## if 'none' then there will be no filter applied
print('trailing days: ' + str(trailing_days))
print('single chain: ' + str(single_chain))

trailing days: 60
single chain: none


In [6]:
chain_config = [
    ### #blockchain, display_name, count_func, count_block_func_param, transactions_table_param, tx_fee_func_param, tx_fee_currency_param, chain_id_param
    ['bitcoin','Bitcoin','COUNT_IF(fee > 0)','COUNT(DISTINCT block_height)','transactions','fee','BTC','NULL'],
    ['near','Near','COUNT(distinct case when gas_price > 0 then tx_hash else null end)','COUNT(DISTINCT block_height)','actions','cast(NULL as double)','NULL','397'],
    ['aptos','Aptos','COUNT_IF(gas_used>0)','COUNT(DISTINCT block_height)','user_transactions','(gas_used*gas_unit_price/1e8)','APT','NULL'],
    # ['stellar','Stellar','COUNT_IF(fee_charged>0)','COUNT(DISTINCT block_height)','fee_charged','history_transactions'], --not date partitioned
    ['kaia','Kaia','COUNT_IF(gas_price>0)','COUNT(DISTINCT block_number)','transactions','cast(NULL as double)','NULL','8217'],
    ['ton','TON','COUNT_IF(compute_gas_fees>0)','COUNT(DISTINCT block_seqno)','transactions','cast(NULL as double)','NULL','NULL'],
    ['berachain','Berachain','COUNT_IF(gas_price>0)','COUNT(DISTINCT block_number)','transactions','cast(gas_price/1e9*gas_used/1e9 as double)','BERA','80094'],
    ['hyperevm','Hyperliquid L1','COUNT_IF(gas_price>0)','COUNT(DISTINCT block_number)','transactions','cast(gas_price/1e9*gas_used/1e9 as double)','HYPE','999'],
    ['sonic','Sonic','COUNT_IF(gas_price>0)','COUNT(DISTINCT block_number)','transactions','cast(gas_price/1e9*gas_used/1e9 as double)','S','146'],
]

In [7]:
days_param = d.generate_query_parameter(input=trailing_days,field_name='trailing_days',dtype='number')
end_days_param = d.generate_query_parameter(input=ending_days,field_name='ending_days',dtype='number')
single_chain_param = d.generate_query_parameter(input=single_chain,field_name='single_chain',dtype='text')

In [8]:
if run_all_txs == 1:
    # Run Dune - all txs
    print('     dune runs')
    # For the main Dune queries, you could wrap them in try-except:
    try:
        fees_df = d.get_dune_data(query_id = 4229341, #https://dune.com/queries/4229341
            name = query_name,
            path = "outputs",
            performance="large",
            params = [days_param,end_days_param,single_chain_param],
            num_hours_to_rerun=rerun_hrs
        )
    except Exception as e:
        print(f"Dune API failed: {e}")
        fees_df = pd.DataFrame()  # Create empty DataFrame as fallback
else:
    fees_df = pd.DataFrame()


     dune runs
Results available at https://dune.com/queries/4229341?trailing_days=60&ending_days=0&single_chain=none
getting latest result


2025-09-23 12:53:33,861 ERROR dune_client.models Can't build ResultsResponse from {'error': 'not found: No execution found for the latest version of the given query'} due to KeyError: 'execution_id'
2025-09-23 12:53:33,862 INFO dune_client.api.base executing 4229341 on large cluster
2025-09-23 12:53:34,222 INFO dune_client.api.base waiting for query execution 01K5VQXAPGQB92H6W06KDFBTPA to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 12:53:35,382 INFO dune_client.api.base waiting for query execution 01K5VQXAPGQB92H6W06KDFBTPA to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 12:53:36,538 INFO dune_client.api.base waiting for query execution 01K5VQXAPGQB92H6W06KDFBTPA to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 12:53:37,699 INFO dune_client.api.base waiting for query execution 01K5VQXAPGQB92H6W06KDFBTPA to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 12:53:38,860 INFO dune_client.api.base waiting fo

In [9]:
if run_gas == 1:
    print('run dune - gas level')
    gas_df = d.get_dune_data(query_id = 4758295, #https://dune.com/queries/4758295
        name = query_name,
        path = "outputs",
        performance="large",
        params = [days_param,end_days_param,single_chain_param],
        num_hours_to_rerun=rerun_hrs
    )
else:
    gas_df = pd.DataFrame()

run dune - gas level
Results available at https://dune.com/queries/4758295?trailing_days=60&ending_days=0&single_chain=none
getting latest result


2025-09-23 13:46:32,699 ERROR dune_client.models Can't build ResultsResponse from {'error': 'not found: No execution found for the latest version of the given query'} due to KeyError: 'execution_id'
2025-09-23 13:46:32,700 INFO dune_client.api.base executing 4758295 on large cluster
2025-09-23 13:46:33,058 INFO dune_client.api.base waiting for query execution 01K5VTYB0G5EKZ19QRAPYHAN80 to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:46:34,242 INFO dune_client.api.base waiting for query execution 01K5VTYB0G5EKZ19QRAPYHAN80 to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:46:35,512 INFO dune_client.api.base waiting for query execution 01K5VTYB0G5EKZ19QRAPYHAN80 to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:46:36,747 INFO dune_client.api.base waiting for query execution 01K5VTYB0G5EKZ19QRAPYHAN80 to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:46:40,023 INFO dune_client.api.base waiting fo

In [10]:
if (run_all_txs == 1) & (single_chain == 'none'):
    fees_df.sample()
    unique_blockchains = fees_df['blockchain'].unique().tolist()
else:
    unique_blockchains = []

In [11]:
if run_metrics == 1:
    print('run dune - fees metrics')
    fee_metrics_df = d.get_dune_data(query_id = 4902405, #https://dune.com/queries/4902405
        name = "dune fees metrics",
        path = "outputs",
        performance="large",
        params = [days_param, end_days_param],
        num_hours_to_rerun=rerun_hrs
    )
    if single_chain != 'none':
        fee_metrics_df = fee_metrics_df[fee_metrics_df['blockchain'] == single_chain]
else:
    fee_metrics_df = pd.DataFrame()

run dune - fees metrics
Results available at https://dune.com/queries/4902405?trailing_days=60&ending_days=0
getting latest result


2025-09-23 13:48:15,958 ERROR dune_client.models Can't build ResultsResponse from {'error': 'not found: No execution found for the latest version of the given query'} due to KeyError: 'execution_id'
2025-09-23 13:48:15,958 INFO dune_client.api.base executing 4902405 on large cluster
2025-09-23 13:48:16,284 INFO dune_client.api.base waiting for query execution 01K5VV1FV86GDE6B6ZZ29FDRND to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:48:17,449 INFO dune_client.api.base waiting for query execution 01K5VV1FV86GDE6B6ZZ29FDRND to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:48:18,608 INFO dune_client.api.base waiting for query execution 01K5VV1FV86GDE6B6ZZ29FDRND to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:48:19,767 INFO dune_client.api.base waiting for query execution 01K5VV1FV86GDE6B6ZZ29FDRND to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:48:20,922 INFO dune_client.api.base waiting fo

In [12]:
chain_df_agg = pd.DataFrame()

In [13]:
if run_chain_level == 1:
    print('run dune - chain level')
    chain_df_agg = pd.DataFrame()
    chain_dfs = []
    for row in chain_config:
        blockchain = row[0]
        print(blockchain)
        if blockchain in unique_blockchains:
            continue
        else:
            #blockchain, display_name, count_func, gas_field, transactions_table
            blockchain_param = d.generate_query_parameter(input=blockchain,field_name='blockchain',dtype='text')
            display_name_param = d.generate_query_parameter(input=row[1],field_name='display_name',dtype='text')
            count_func_param = d.generate_query_parameter(input=row[2],field_name='count_func',dtype='text')
            count_block_func_param = d.generate_query_parameter(input=row[3],field_name='count_block_func',dtype='text')
            # gas_field_param = d.generate_query_parameter(input=row[4],field_name='gas_field',dtype='text')
            transactions_table_param = d.generate_query_parameter(input=row[4],field_name='transactions_table',dtype='text')
            tx_fee_func_param = d.generate_query_parameter(input=row[5],field_name='tx_fee_func_internal',dtype='text')
            tx_fee_currency_param = d.generate_query_parameter(input=row[6],field_name='tx_fee_currency',dtype='text')
            chain_id_param = d.generate_query_parameter(input=row[7],field_name='chain_id_param',dtype='text')

            chain_df = d.get_dune_data(query_id = 4230061, #https://dune.com/queries/4230061
                name = query_name + '_by_chain',
                path = "outputs",
                performance="large",
                params = [
                        days_param,end_days_param,blockchain_param,display_name_param,count_func_param,count_block_func_param
                        ,transactions_table_param,tx_fee_func_param,tx_fee_currency_param,chain_id_param
                        ],
                num_hours_to_rerun=rerun_hrs
            )
            # print(chain_df.sample(3))
            chain_dfs.append(chain_df)

    chain_df_agg = pd.concat(chain_dfs)
else:
    chain_df_agg = pd.DataFrame()

run dune - chain level
bitcoin
Results available at https://dune.com/queries/4230061?trailing_days=60&ending_days=0&blockchain=bitcoin&display_name=Bitcoin&count_func=COUNT_IF%28fee+%3E+0%29&count_block_func=COUNT%28DISTINCT+block_height%29&transactions_table=transactions&tx_fee_func_internal=fee&tx_fee_currency=BTC&chain_id_param=NULL
getting latest result


2025-09-23 13:48:26,694 ERROR dune_client.models Can't build ResultsResponse from {'error': 'not found: No execution found for the latest version of the given query'} due to KeyError: 'execution_id'
2025-09-23 13:48:26,695 INFO dune_client.api.base executing 4230061 on large cluster
2025-09-23 13:48:27,026 INFO dune_client.api.base waiting for query execution 01K5VV1TB0THVZJXNMEXFZBA9B to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:48:28,183 INFO dune_client.api.base waiting for query execution 01K5VV1TB0THVZJXNMEXFZBA9B to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:48:29,338 INFO dune_client.api.base waiting for query execution 01K5VV1TB0THVZJXNMEXFZBA9B to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:48:30,502 INFO dune_client.api.base waiting for query execution 01K5VV1TB0THVZJXNMEXFZBA9B to complete: ExecutionState.EXECUTING
2025-09-23 13:48:31,659 INFO dune_client.api.base waiting for query execution 01K

near
Results available at https://dune.com/queries/4230061?trailing_days=60&ending_days=0&blockchain=near&display_name=Near&count_func=COUNT%28distinct+case+when+gas_price+%3E+0+then+tx_hash+else+null+end%29&count_block_func=COUNT%28DISTINCT+block_height%29&transactions_table=actions&tx_fee_func_internal=cast%28NULL+as+double%29&tx_fee_currency=NULL&chain_id_param=397
getting latest result


2025-09-23 13:48:42,804 ERROR dune_client.models Can't build ResultsResponse from {'error': 'not found: No execution found for the latest version of the given query'} due to KeyError: 'execution_id'
2025-09-23 13:48:42,805 INFO dune_client.api.base executing 4230061 on large cluster
2025-09-23 13:48:43,121 INFO dune_client.api.base waiting for query execution 01K5VV2A23RD13Y65JYT8RH1AZ to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:48:44,281 INFO dune_client.api.base waiting for query execution 01K5VV2A23RD13Y65JYT8RH1AZ to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:48:45,439 INFO dune_client.api.base waiting for query execution 01K5VV2A23RD13Y65JYT8RH1AZ to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:48:46,587 INFO dune_client.api.base waiting for query execution 01K5VV2A23RD13Y65JYT8RH1AZ to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:48:47,739 INFO dune_client.api.base waiting fo

aptos
Results available at https://dune.com/queries/4230061?trailing_days=60&ending_days=0&blockchain=aptos&display_name=Aptos&count_func=COUNT_IF%28gas_used%3E0%29&count_block_func=COUNT%28DISTINCT+block_height%29&transactions_table=user_transactions&tx_fee_func_internal=%28gas_used%2Agas_unit_price%2F1e8%29&tx_fee_currency=APT&chain_id_param=NULL
getting latest result


...

2025-09-23 13:53:28,134 ERROR dune_client.models Can't build ResultsResponse from {'error': 'not found: No execution found for the latest version of the given query'} due to KeyError: 'execution_id'
2025-09-23 13:53:28,135 INFO dune_client.api.base executing 4230061 on large cluster
2025-09-23 13:53:28,464 INFO dune_client.api.base waiting for query execution 01K5VVB0Q5B3EEWH0W6M86AJTC to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:53:29,613 INFO dune_client.api.base waiting for query execution 01K5VVB0Q5B3EEWH0W6M86AJTC to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:53:30,772 INFO dune_client.api.base waiting for query execution 01K5VVB0Q5B3EEWH0W6M86AJTC to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:53:31,929 INFO dune_client.api.base waiting for query execution 01K5VVB0Q5B3EEWH0W6M86AJTC to complete: ExecutionState.EXECUTING
2025-09-23 13:53:33,477 INFO dune_client.api.base waiting for query execution 01K

berachain
hyperevm
Results available at https://dune.com/queries/4230061?trailing_days=60&ending_days=0&blockchain=hyperevm&display_name=Hyperliquid+L1&count_func=COUNT_IF%28gas_price%3E0%29&count_block_func=COUNT%28DISTINCT+block_number%29&transactions_table=transactions&tx_fee_func_internal=cast%28gas_price%2F1e9%2Agas_used%2F1e9+as+double%29&tx_fee_currency=HYPE&chain_id_param=999
getting latest result


2025-09-23 13:53:36,525 ERROR dune_client.models Can't build ResultsResponse from {'error': 'not found: No execution found for the latest version of the given query'} due to KeyError: 'execution_id'
2025-09-23 13:53:36,526 INFO dune_client.api.base executing 4230061 on large cluster
2025-09-23 13:53:36,851 INFO dune_client.api.base waiting for query execution 01K5VVB8X0FGZW1QZAMBHNXWAQ to complete: ExecutionState.PENDING (queue position: None)
2025-09-23 13:53:38,018 INFO dune_client.api.base waiting for query execution 01K5VVB8X0FGZW1QZAMBHNXWAQ to complete: ExecutionState.EXECUTING
2025-09-23 13:53:39,184 INFO dune_client.api.base waiting for query execution 01K5VVB8X0FGZW1QZAMBHNXWAQ to complete: ExecutionState.EXECUTING
2025-09-23 13:53:40,346 INFO dune_client.api.base waiting for query execution 01K5VVB8X0FGZW1QZAMBHNXWAQ to complete: ExecutionState.EXECUTING
2025-09-23 13:53:41,504 INFO dune_client.api.base waiting for query execution 01K5VVB8X0FGZW1QZAMBHNXWAQ to complete: Execu

sonic


/var/folders/nl/qcdk9wr515b2h3s1kd4z08w80000gn/T/ipykernel_20141/1239014147.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  chain_df_agg = pd.concat(chain_dfs)


In [14]:
print(fees_df.columns)
# print(chain_df_agg.columns)

Index(['blockchain', 'chain_id', 'display_name', 'dt', 'median_tx_fee_native',
       'median_tx_fee_usd', 'num_blocks', 'num_txs', 'tx_fee_currency',
       'tx_fee_native', 'tx_fee_usd', 'last_updated'],
      dtype='object')


In [15]:
if not chain_df_agg.empty:
    dune_df = pd.concat([fees_df,chain_df_agg])
else: 
    dune_df = fees_df.copy()

if run_metrics == 1:
# First, find chains that are only in metric_fees_df
    unique_fee_metrics_df = fee_metrics_df[~fee_metrics_df['blockchain'].isin(dune_df['blockchain'])]
    dune_df = pd.concat([dune_df, unique_fee_metrics_df])

In [16]:
dune_df

,blockchain,chain_id,display_name,dt,median_tx_fee_native,median_tx_fee_usd,num_blocks,num_txs,tx_fee_currency,tx_fee_native,tx_fee_usd,last_updated
0,celo,42220.0,Celo,2025-08-29,0.001539,0.001540,86400.0,503787.0,0x48065fbbe25f71c9282ddf5e1cd6d6a887483d5e,707.848125,708.323057,2025-09-23 16:53:34.032865
1,zksync,324.0,zkSync Era,2025-09-06,0.000005,0.022203,16413.0,5164.0,0x000000000000000000000000000000000000800a,0.049532,212.804802,2025-09-23 16:53:34.032865
2,fantom,250.0,Fantom,2025-09-06,0.000123,0.000037,24291.0,40603.0,0x0000000000000000000000000000000000000000,36.463795,10.983247,2025-09-23 16:53:34.032865
3,kaia,8217.0,Kaia,2025-08-29,0.000655,0.000099,86306.0,1135924.0,0x0000000000000000000000000000000000000000,3239.534044,486.962204,2025-09-23 16:53:34.032865
4,kaia,8217.0,Kaia,2025-09-06,0.000655,0.000095,86070.0,962338.0,0x0000000000000000000000000000000000000000,2671.439867,389.302872,2025-09-23 16:53:34.032865
...,...,...,...,...,...,...,...,...,...,...,...,...
51,xrpl,NaN,Xrpl,2025-08-24 00:00:00.000 UTC,NaN,NaN,NaN,NaN,NaN,NaN,910.699172,2025-09-23 17:48:16.105374
52,xrpl,NaN,Xrpl,2025-08-25 00:00:00.000 UTC,NaN,NaN,NaN,NaN,NaN,NaN,1331.964460,2025-09-23 17:48:16.105374
53,xrpl,NaN,Xrpl,2025-08-26 00:00:00.000 UTC,NaN,NaN,NaN,NaN,NaN,NaN,1318.313977,2025-09-23 17:48:16.105374
54,xrpl,NaN,Xrpl,2025-08-18 00:00:00.000 UTC,NaN,NaN,NaN,NaN,NaN,NaN,6887.605188,2025-09-23 17:48:16.105374


In [17]:
if not dune_df.empty:
    dune_df['source'] = 'dune'
    dune_df['dt'] = dune_df['dt'].str.replace(' UTC', '', regex=False)
    dune_df['dt'] = pd.to_datetime(dune_df['dt'], format='mixed').dt.tz_localize(None)
    dune_df = dune_df[dune_df['dt'].dt.date < datetime.datetime.now(datetime.timezone.utc).date()]
    dune_df['chain_id'] = dune_df['chain_id'].astype(str)
    dune_df['chain_id'] = dune_df['chain_id'].astype(str).str.replace(r'\.0$', '', regex=True)
    dune_df['num_blocks'] = dune_df['num_blocks'].fillna(0).astype('Int64')
    dune_df['num_txs'] = dune_df['num_txs'].fillna(0).astype('Int64')

/var/folders/nl/qcdk9wr515b2h3s1kd4z08w80000gn/T/ipykernel_20141/539739434.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dune_df['chain_id'] = dune_df['chain_id'].astype(str)
/var/folders/nl/qcdk9wr515b2h3s1kd4z08w80000gn/T/ipykernel_20141/539739434.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dune_df['chain_id'] = dune_df['chain_id'].astype(str).str.replace(r'\.0$', '', regex=True)
/var/folders/nl/qcdk9wr515b2h3s1kd4z08w80000gn/T/ipykernel_20141/539739434.py:8: SettingWithCopyWarning: 
A value

In [18]:
if not gas_df.empty:
    gas_df['source'] = 'dune'
    gas_df['dt'] = pd.to_datetime(gas_df['dt']).dt.tz_localize(None)
    gas_df = gas_df[gas_df['dt'].dt.date < datetime.datetime.now(datetime.timezone.utc).date()]
    gas_df['chain_id'] = gas_df['chain_id'].astype(str)
    gas_df['chain_id'] = gas_df['chain_id'].astype(str).str.replace(r'\.0$', '', regex=True)

In [19]:
# Verify that all elements are strings
assert dune_df['chain_id'].apply(type).eq(str).all(), "Not all elements are strings"
# print(dune_df['chain_id'].dtype)

In [20]:
if not fee_metrics_df.empty:
    fee_metrics_df['dt'] = pd.to_datetime(fee_metrics_df['dt']).dt.tz_localize(None)
    fee_metrics_df['chain_id'] = fee_metrics_df['chain_id'].astype(str)

In [21]:
if not gas_df.empty:
    assert gas_df['chain_id'].apply(type).eq(str).all(), "Not all elements are strings"
if not fee_metrics_df.empty:
    assert fee_metrics_df['chain_id'].apply(type).eq(str).all(), "Not all elements are strings"

In [22]:
# dune_df.sample(5)
print(dune_df.dtypes)


blockchain                      object
chain_id                        object
display_name                    object
dt                      datetime64[ns]
median_tx_fee_native           float64
median_tx_fee_usd              float64
num_blocks                       Int64
num_txs                          Int64
tx_fee_currency                 object
tx_fee_native                  float64
tx_fee_usd                     float64
last_updated            datetime64[ns]
source                          object
dtype: object


In [23]:
print(f"dune_df shape: {dune_df.shape}")

dune_df shape: (3433, 13)


In [24]:
unique_cols = ['blockchain', 'dt','tx_fee_currency']

In [25]:
# Identify duplicate combinations
duplicates = dune_df.duplicated(subset=unique_cols, keep=False)

# View the duplicate rows
duplicate_rows = dune_df[duplicates]

# Display the duplicate rows
print(duplicate_rows)

# Get a count of duplicates for each combination
duplicate_counts = dune_df.groupby(unique_cols).size().reset_index(name='count')
duplicate_counts = duplicate_counts[duplicate_counts['count'] > 1]

print("\nDuplicate combination counts:")
print(duplicate_counts)

Empty DataFrame
Columns: [blockchain, chain_id, display_name, dt, median_tx_fee_native, median_tx_fee_usd, num_blocks, num_txs, tx_fee_currency, tx_fee_native, tx_fee_usd, last_updated, source]
Index: []

Duplicate combination counts:
Empty DataFrame
Columns: [blockchain, dt, tx_fee_currency, count]
Index: []


In [26]:
dune_df.dtypes

blockchain                      object
chain_id                        object
display_name                    object
dt                      datetime64[ns]
median_tx_fee_native           float64
median_tx_fee_usd              float64
num_blocks                       Int64
num_txs                          Int64
tx_fee_currency                 object
tx_fee_native                  float64
tx_fee_usd                     float64
last_updated            datetime64[ns]
source                          object
dtype: object

In [27]:
dune_df.dtypes

blockchain                      object
chain_id                        object
display_name                    object
dt                      datetime64[ns]
median_tx_fee_native           float64
median_tx_fee_usd              float64
num_blocks                       Int64
num_txs                          Int64
tx_fee_currency                 object
tx_fee_native                  float64
tx_fee_usd                     float64
last_updated            datetime64[ns]
source                          object
dtype: object

In [28]:
dune_df['num_blocks'] = pd.to_numeric(dune_df['num_blocks'], errors='coerce').fillna(0).round().astype(int)
dune_df['num_txs'] = pd.to_numeric(dune_df['num_txs'], errors='coerce').fillna(0).round().astype(int)

/var/folders/nl/qcdk9wr515b2h3s1kd4z08w80000gn/T/ipykernel_20141/3922221742.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dune_df['num_blocks'] = pd.to_numeric(dune_df['num_blocks'], errors='coerce').fillna(0).round().astype(int)
/var/folders/nl/qcdk9wr515b2h3s1kd4z08w80000gn/T/ipykernel_20141/3922221742.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dune_df['num_txs'] = pd.to_numeric(dune_df['num_txs'], errors='coerce').fillna(0).round().astype(int)


In [29]:
#Check
if not dune_df.empty:
    print(dune_df.sample(5))

     blockchain chain_id display_name         dt  median_tx_fee_native  \
432      solana      nan       Solana 2025-08-18              0.000006   
5           sei     1329  Sei Network 2025-09-06              0.000062   
140         bob    60808          BOB 2025-07-29              0.000016   
17         near      397         Near 2025-09-22                   NaN   
1601   abstract     2741     Abstract 2025-08-15                   NaN   

      median_tx_fee_usd  num_blocks   num_txs  \
432            0.001134      219572  91098708   
5              0.000018      175557   1848159   
140            0.060632       43200      8493   
17                  NaN      140546   4547817   
1601                NaN       83644         0   

                                  tx_fee_currency  tx_fee_native  \
432   So11111111111111111111111111111111111111112    6270.269358   
5      0x0000000000000000000000000000000000000000     473.126891   
140    0x0000000000000000000000000000000000000000       

In [30]:
#BQ Upload
if not dune_df.empty:
    bqu.append_and_upsert_df_to_bq_table(dune_df, query_name, unique_keys = unique_cols)

2025-09-23 13:53:46,597 INFO root Attempt 1: Using OIDC login
/Users/michael/Documents/GitHub/op-analytics/other_chains_tracking/../helper_functions/google_bq_utils.py:373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[key].fillna('none')
2025-09-23 13:53:48,231 ERROR google_bq_utils An error occurred while checking table existence
2025-09-23 13:53:48,232 ERROR google_bq_utils Error during append_and_upsert_df_to_bq_table on attempt 1: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.


Waiting before retry...


2025-09-23 13:53:49,235 INFO root Attempt 1: Using OIDC login
/Users/michael/Documents/GitHub/op-analytics/other_chains_tracking/../helper_functions/google_bq_utils.py:373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[key].fillna('none')
2025-09-23 13:53:49,753 ERROR google_bq_utils An error occurred while checking table existence
2025-09-23 13:53:49,754 ERROR google_bq_utils Error during append_and_upsert_df_to_bq_table on attempt 2: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.


Waiting before retry...


2025-09-23 13:53:51,755 INFO root Attempt 1: Using OIDC login
/Users/michael/Documents/GitHub/op-analytics/other_chains_tracking/../helper_functions/google_bq_utils.py:373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[key].fillna('none')
2025-09-23 13:53:52,281 ERROR google_bq_utils An error occurred while checking table existence
2025-09-23 13:53:52,281 ERROR google_bq_utils Error during append_and_upsert_df_to_bq_table on attempt 3: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.


Error on attempt 3: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.
All attempts failed


RefreshError: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.

In [ ]:
if not gas_df.empty:
    bqu.append_and_upsert_df_to_bq_table(gas_df, query_gas_name, unique_keys = ['chain_id','dt'])

In [ ]:
# fee_metrics_df[fee_metrics_df['blockchain']=='tron'].sort_values(by='dt',ascending=False)

In [ ]:
if not fee_metrics_df.empty:
    bqu.append_and_upsert_df_to_bq_table(fee_metrics_df, query_fees_name, unique_keys = ['blockchain','dt'])